## Spark

In the same way that for many programs hellow word is the first program in spark is computing pi


In [1]:
import random
NUM_SAMPLES = 100000000
def inside(p):
 x, y = random.random(), random.random()
 return x*x + y*y < 1
count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
pi = 4 * count / NUM_SAMPLES
print("Pi is roughly", pi)

NameError: name 'sc' is not defined

## SQL and DataFrames

There are two approaches to Spark, the DataFrame approach and the RDD approach. We are going to learn the SQL approach since it is works in the way spark intends.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [10]:
spark = SparkSession.builder.appName('iris_clf').getOrCreate()

In [20]:
df = spark.read.csv('iris.csv',header=True,inferSchema=True)
df.show(5)

+------------+-----------+------------+-----------+------+
|sepal_length|sepal_width|petal_length|petal_width|  type|
+------------+-----------+------------+-----------+------+
|         5.1|        3.5|         1.4|        0.2|Setosa|
|         4.9|        3.0|         1.4|        0.2|Setosa|
|         4.7|        3.2|         1.3|        0.2|Setosa|
|         4.6|        3.1|         1.5|        0.2|Setosa|
|         5.0|        3.6|         1.4|        0.2|Setosa|
+------------+-----------+------------+-----------+------+
only showing top 5 rows



In [21]:
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- type: string (nullable = true)



In [22]:
schema = StructType([
    StructField('sepal_length',DoubleType()),
    StructField('sepal_width',DoubleType()),
    StructField('petal_length',DoubleType()),
    StructField('petal_width',DoubleType()),
    StructField('type',StringType())
    ])

In [23]:
df2 = spark.read.csv('iris.csv',header=True,schema=schema)
df2.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- type: string (nullable = true)



In [24]:
from numpy import vectorize


input_col = ['sepal_length','sepal_width','petal_length', 'petal_width']
vectorizer = VectorAssembler(inputCols=input_col, outputCol='features')

df = vectorizer.transform(df)
df.show(5)

+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|  type|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         5.1|        3.5|         1.4|        0.2|Setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|Setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|Setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|Setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|Setosa|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+------+-----------------+
only showing top 5 rows



In [27]:
indexer = StringIndexer(inputCol='type', outputCol='indexed_type')
df = indexer.fit(df).transform(df)
df.show(5)

+------------+-----------+------------+-----------+------+-----------------+------------+
|sepal_length|sepal_width|petal_length|petal_width|  type|         features|indexed_type|
+------------+-----------+------------+-----------+------+-----------------+------------+
|         5.1|        3.5|         1.4|        0.2|Setosa|[5.1,3.5,1.4,0.2]|         0.0|
|         4.9|        3.0|         1.4|        0.2|Setosa|[4.9,3.0,1.4,0.2]|         0.0|
|         4.7|        3.2|         1.3|        0.2|Setosa|[4.7,3.2,1.3,0.2]|         0.0|
|         4.6|        3.1|         1.5|        0.2|Setosa|[4.6,3.1,1.5,0.2]|         0.0|
|         5.0|        3.6|         1.4|        0.2|Setosa|[5.0,3.6,1.4,0.2]|         0.0|
+------------+-----------+------------+-----------+------+-----------------+------------+
only showing top 5 rows



In [37]:
df_train, df_test = df.randomSplit([0.8,0.2], seed=0)
df_train.show(5)

+------------+-----------+------------+-----------+------+-----------------+------------+
|sepal_length|sepal_width|petal_length|petal_width|  type|         features|indexed_type|
+------------+-----------+------------+-----------+------+-----------------+------------+
|         4.3|        3.0|         1.1|        0.1|Setosa|[4.3,3.0,1.1,0.1]|         0.0|
|         4.4|        2.9|         1.4|        0.2|Setosa|[4.4,2.9,1.4,0.2]|         0.0|
|         4.4|        3.0|         1.3|        0.2|Setosa|[4.4,3.0,1.3,0.2]|         0.0|
|         4.4|        3.2|         1.3|        0.2|Setosa|[4.4,3.2,1.3,0.2]|         0.0|
|         4.5|        2.3|         1.3|        0.3|Setosa|[4.5,2.3,1.3,0.3]|         0.0|
+------------+-----------+------------+-----------+------+-----------------+------------+
only showing top 5 rows



In [35]:
# df_train,df_val, df_test = df.randomSplit([0.7,0.1,0.2], seed=0)
# df_val.show()

In [38]:
rf_clf = RandomForestClassifier(featuresCol='features',labelCol='indexed_type')
rf_clf = rf_clf.fit(df_train)
df_test = rf_clf.transform(df_test)
df_test.show(5)

+------------+-----------+------------+-----------+------+-----------------+------------+--------------+-------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|  type|         features|indexed_type| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+------+-----------------+------------+--------------+-------------+----------+
|         4.9|        3.1|         1.5|        0.2|Setosa|[4.9,3.1,1.5,0.2]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.9|        3.6|         1.4|        0.1|Setosa|[4.9,3.6,1.4,0.1]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         5.0|        3.4|         1.6|        0.4|Setosa|[5.0,3.4,1.6,0.4]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         5.0|        3.5|         1.3|        0.3|Setosa|[5.0,3.5,1.3,0.3]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         5.0|        3.5|         1.6|        0.6|Setosa|[5.0,3.5,1.6,0.6]|         0.0|[

In [41]:
df_test.select('type','features','indexed_type','rawPrediction','probability','prediction').show(5)

+------+-----------------+------------+--------------+-------------+----------+
|  type|         features|indexed_type| rawPrediction|  probability|prediction|
+------+-----------------+------------+--------------+-------------+----------+
|Setosa|[4.9,3.1,1.5,0.2]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|Setosa|[4.9,3.6,1.4,0.1]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|Setosa|[5.0,3.4,1.6,0.4]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|Setosa|[5.0,3.5,1.3,0.3]|         0.0|[20.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|Setosa|[5.0,3.5,1.6,0.6]|         0.0|[16.0,4.0,0.0]|[0.8,0.2,0.0]|       0.0|
+------+-----------------+------------+--------------+-------------+----------+
only showing top 5 rows



In [43]:
criterion = MulticlassClassificationEvaluator(labelCol='indexed_type',predictionCol='prediction')
accuracy = criterion.evaluate(df_test)
accuracy

1.0

## Session

Spark needs to use a session in order to process data in a parallel way.

A session can be built in many different ways, what is going to be a difference maker for most local machines is that we need to specify to spark to either get it or create it.

We will use this session to define our Spark DataFrames.

When Creating DataFrames we can let spark infer the schema.

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- type: string (nullable = true)



Or we can create our own schema

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- type: string (nullable = true)



## Features

Spark doesn`t need the X and Y separated in the standard format.

As you will see throught the notebook spark will directly operate on the DataFrame after specifying an input and aoutput column.

By default Spark takes a column called deatures as the input in all classifiers and the Y column is called labels

We can create the  feature column by using a vector assembler

In [26]:
numericCols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']


+------------+-----------+------------+-----------+------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|  type|         features|
+------------+-----------+------------+-----------+------+-----------------+
|         5.1|        3.5|         1.4|        0.2|Setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|Setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|Setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|Setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|Setosa|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4|Setosa|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3|Setosa|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2|Setosa|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2|Setosa|[4.4,2.9,1.4,0.2]|
|         4.9|        3.1|         1.5|        0.1|Setosa|[4.9,3.1,1.5,0.1]|

## Encoding

We can use a string indexes in the same way as the vector assembler to ordinally encode our types

+------------+-----------+------------+-----------+------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|  type|         features|labelIndex|
+------------+-----------+------------+-----------+------+-----------------+----------+
|         5.1|        3.5|         1.4|        0.2|Setosa|[5.1,3.5,1.4,0.2]|       0.0|
|         4.9|        3.0|         1.4|        0.2|Setosa|[4.9,3.0,1.4,0.2]|       0.0|
|         4.7|        3.2|         1.3|        0.2|Setosa|[4.7,3.2,1.3,0.2]|       0.0|
|         4.6|        3.1|         1.5|        0.2|Setosa|[4.6,3.1,1.5,0.2]|       0.0|
|         5.0|        3.6|         1.4|        0.2|Setosa|[5.0,3.6,1.4,0.2]|       0.0|
|         5.4|        3.9|         1.7|        0.4|Setosa|[5.4,3.9,1.7,0.4]|       0.0|
|         4.6|        3.4|         1.4|        0.3|Setosa|[4.6,3.4,1.4,0.3]|       0.0|
|         5.0|        3.4|         1.5|        0.2|Setosa|[5.0,3.4,1.5,0.2]|       0.0|
|         4.4|        2.9|      

## Train/Test split

Spark Dataframes come pre-equipped with a random split function that will give you as many portions as specified.

The proportions for each portion are passed in a list

## Classifiers

Many Spark classifiers unfortunatelly do not handle good multylabel classification so be very carefull with which you choose.

they can all be found here: https://spark.apache.org/docs/latest/ml-classification-regression.html

## Specifying input and target

As I said the default names are features an label, but we can also specify them

## Fitting

It is done in the same way as SKlearn

## Predictions
    
This part is a little different. Spark will not output a prediction vector, it will direclty add a column to the DataFrame.

To predict we call the method 'transform' from the classifier

## Selecting

This structures are built to be parallelized in the CPU so we cannot access them in a standard fashion.

To get a subset of columns we need to use select

+------------+-----------+------------+-----------+----------+--------------------+----------+--------------------+
|sepal_length|sepal_width|petal_length|petal_width|labelIndex|       rawPrediction|prediction|         probability|
+------------+-----------+------------+-----------+----------+--------------------+----------+--------------------+
|         4.3|        3.0|         1.1|        0.1|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         4.5|        2.3|         1.3|        0.3|       0.0|      [19.0,0.0,1.0]|       0.0|     [0.95,0.0,0.05]|
|         4.6|        3.6|         1.0|        0.2|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         4.9|        3.1|         1.5|        0.2|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         4.9|        3.6|         1.4|        0.1|       0.0|      [20.0,0.0,0.0]|       0.0|       [1.0,0.0,0.0]|
|         5.0|        3.4|         1.6|        0.4|       0.0|      [19.

+----------+----------+
|labelIndex|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 10 rows



## Evaluating

In a very familiar format 

Accuracy = 0.9754807071880243
Test Error = 0.024519292811975735
